In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.callbacks import TensorBoard
DATADIR="C:/Users/huang/Desktop/2019 Summer/kagglecatsanddogs_3367a/PetImages"

In [2]:
CATEGORIES=["Dog","Cat"]
import time
Name="Cats-vs-dog-cnn-{}".format(int(time.time()))
tensorboard=TensorBoard(log_dir=r'C:\Users\huang\Desktop\log\{}'.format(Name))

In [3]:
IMG_SIZE=50

In [4]:
training_data=[]
def create_training_data():
    for category in CATEGORIES:
        path=os.path.join(DATADIR,category)
        class_num=CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
                new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                training_data.append([new_array,class_num])
            except Exception as e:
                pass
create_training_data()

In [5]:
len(training_data)

24946

In [6]:
for sample in training_data[:20]:
    print(sample[1])

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [7]:
import random
random.shuffle(training_data)
for sample in training_data[:20]:
    print(sample[1])

0
1
1
0
1
0
0
1
0
0
1
1
1
1
0
1
0
0
0
0


In [8]:
X=[]
y=[]
unpack=[x for [a,x] in training_data]

In [9]:
unique_elements,count_elements=np.unique(unpack,return_counts=True)
print(np.asarray((unique_elements,count_elements)))

[[    0     1]
 [12470 12476]]


In [10]:
for features,label in training_data:
    X.append(features)
    y.append(label)


In [11]:
for a in range(1):
    print(X[a])
X[0].shape

[[ 97  94  60 ... 157 190 126]
 [100  96  61 ... 181 131 108]
 [ 99  95  63 ... 110 149 104]
 ...
 [ 78  77  78 ... 150 146 127]
 [ 75  78  76 ... 155 151 138]
 [ 73  73  76 ... 150 151 141]]


(50, 50)

In [18]:
X_cp=X
X_cp=np.array(X_cp).reshape(-1,IMG_SIZE,IMG_SIZE,1)
X_cp[0].shape

(50, 50, 1)

In [16]:
X=np.array(X).reshape(-1,IMG_SIZE,IMG_SIZE,1)

In [17]:
for a in range(1):
    print(X[a])
X[0].shape

[[[ 97]
  [ 94]
  [ 60]
  ...
  [157]
  [190]
  [126]]

 [[100]
  [ 96]
  [ 61]
  ...
  [181]
  [131]
  [108]]

 [[ 99]
  [ 95]
  [ 63]
  ...
  [110]
  [149]
  [104]]

 ...

 [[ 78]
  [ 77]
  [ 78]
  ...
  [150]
  [146]
  [127]]

 [[ 75]
  [ 78]
  [ 76]
  ...
  [155]
  [151]
  [138]]

 [[ 73]
  [ 73]
  [ 76]
  ...
  [150]
  [151]
  [141]]]


(50, 50, 1)

In [14]:
import pickle
pickle_out=open("X_pickle","wb")
pickle.dump(X,pickle_out)
pickle_out.close()

pickle_out=open("y_pickle","wb")
pickle.dump(y,pickle_out)
pickle_out.close()

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,MaxPooling2D



In [16]:
sess=tf.Session()
file_writer = tf.summary.FileWriter(r'C:\Users\huang\Desktop\log',tf.get_default_graph())
file_writer.flush()
file_writer.close()

In [40]:
X=X/255.0
model=Sequential()
model.add(Conv2D(64,(3,3),input_shape=X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
#model.add(Dense(64))
#model.add(Activation("relu"))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
             optimizer="adam",
             metrics=['accuracy'])
model.fit(X,y,batch_size=32,epochs=10,validation_split=0.3,callbacks=[tensorboard])


Train on 17462 samples, validate on 7484 samples
Epoch 1/10
17462/17462 [==============================] - 5s 302us/sample - loss: 0.6933 - acc: 0.5026 - val_loss: 0.6931 - val_acc: 0.5013
Epoch 2/10
17462/17462 [==============================] - 5s 286us/sample - loss: 0.6932 - acc: 0.5014 - val_loss: 0.6931 - val_acc: 0.5013
Epoch 3/10
17462/17462 [==============================] - 5s 285us/sample - loss: 0.6932 - acc: 0.4990 - val_loss: 0.6931 - val_acc: 0.5013
Epoch 4/10
17462/17462 [==============================] - 5s 286us/sample - loss: 0.6932 - acc: 0.4949 - val_loss: 0.6932 - val_acc: 0.4987
Epoch 5/10
17462/17462 [==============================] - 5s 287us/sample - loss: 0.6932 - acc: 0.5019 - val_loss: 0.6931 - val_acc: 0.5013
Epoch 6/10
17462/17462 [==============================] - 5s 287us/sample - loss: 0.6932 - acc: 0.4936 - val_loss: 0.6931 - val_acc: 0.5013
Epoch 7/10
17462/17462 [==============================] - 5s 288us/sample - loss: 0.6932 - acc: 0.4989 - val_lo

In [41]:
X.shape[1:]

(50, 50, 1)

In [42]:
def prepare(filepath):
    IMG_SIZE=50
    img_array=cv2.imread(filepath,cv2.IMREAD_GRAYSCALE)
    new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
    return new_array.reshape(-1,IMG_SIZE,IMG_SIZE,1)
prediction=model.predict(prepare('dog.jpg'))
print(CATEGORIES[int(prediction[0][0])])

Cat


In [43]:
prediction_1=model.predict(prepare('cat.jpg'))
print(CATEGORIES[int(prediction_1[0][0])])

Dog


In [44]:
prediction_2=model.predict(prepare('catcat.jpg'))
print(CATEGORIES[int(prediction_2[0][0])])

Cat
